In [1]:
import os
import sys
import torch

In [2]:
notebook_dir = os.path.dirname(os.path.abspath('source/main.ipynb'))
source_path = os.path.join(notebook_dir)
sys.path.append(source_path)

from classifier.model_architecture import CowClassifier

In [3]:
# Configuración de directorios
SAVE_DIR = '../results/classifier/cut_images'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Cargar de detección
model_detection = torch.hub.load('ultralytics/yolov5', 'custom', '../models/bounding/best.pt')

# Cargar modelo de clasificación
model_classification = CowClassifier().to(device)

# Cargar los pesos en el modelo de clasificación
model_classification.load_state_dict(torch.load('../models/classifier/cow_class_model_state.pth', map_location=device))
model_classification.eval()

Using cache found in C:\Users\Artur/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-31 Python-3.10.11 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


CowClassifier(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=100352, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.5, inplace=False)
    (13): Linear(in_features=128, out_features=64, bias=True)
    (14): ReLU()
    (15): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [ ]:
from model_utils import detect_objects

DIRECTORY = '../dataset/bounding/detect/'

# Ejecutar la detección y obtener los resultados
results = detect_objects(DIRECTORY)

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
